In [1]:
!pip install gdown
!pip install unrar
!pip install imageio

In [2]:
!gdown --id 1Eg9KEUzPpiWZPQF91lHll0a6fcpuGs2h

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1Eg9KEUzPpiWZPQF91lHll0a6fcpuGs2h
To: /mnt/batch/tasks/shared/LS_root/mounts/clusters/asarcpu/code/Users/abdullahmuhamed3604_sd/VML-HD.zip
100%|██████████████████████████████████████| 5.39G/5.39G [01:43<00:00, 51.9MB/s]


In [3]:
!gdown --id 1Cw8CTYxXXbu11JhOF4zSudyEA9RX-rjJ

/anaconda/envs/azureml_py36/lib/python3.6/site-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1Cw8CTYxXXbu11JhOF4zSudyEA9RX-rjJ
To: /mnt/batch/tasks/shared/LS_root/mounts/clusters/asarcpu/code/Users/abdullahmuhamed3604_sd/xmlFiles.rar
100%|██████████████████████████████████████| 3.60M/3.60M [00:00<00:00, 28.9MB/s]


In [4]:
!unzip VML-HD.zip
!unrar x xmlFiles.rar

Archive:  VML-HD.zip
   creating: 3157556/
  inflating: 3157556/0003-1.png      
  inflating: 3157556/0003-2.png      
  inflating: 3157556/0004-1.png      
  inflating: 3157556/0004-2.png      
  inflating: 3157556/0005-1.png      
  inflating: 3157556/0005-2.png      
  inflating: 3157556/0006-1.png      
  inflating: 3157556/0006-2.png      
  inflating: 3157556/0007-1.png      
  inflating: 3157556/0007-2.png      
  inflating: 3157556/0008-1.png      
  inflating: 3157556/0008-2.png      
  inflating: 3157556/0009-1.png      
  inflating: 3157556/0009-2.png      
  inflating: 3157556/0010-1.png      
  inflating: 3157556/0010-2.png      
  inflating: 3157556/0011-1.png      
  inflating: 3157556/0011-2.png      
  inflating: 3157556/0012-1.png      
  inflating: 3157556/0012-2.png      
  inflating: 3157556/0013-1.png      
  inflating: 3157556/0013-2.png      
  inflating: 3157556/0014-1.png      
  inflating: 3157556/0014-2.png      
  inflating: 3157556/0015-1.png      
  infla

# Crop Dataset

In [2]:
import os
import xml.etree.ElementTree as ET
from imageio import imread, imsave

books = ['3157556', '3158466', '3249138', '3368132', '3426930']
root_path='segdata/'
idx = 0

for book in books:
    print('parsing manuscript ', book)
    tree = ET.parse(book + '/docElements.xml')
    root = tree.getroot()
    pad = 4
    labels = []
    idf = 0
    for image in root.iter('image'):
        pg=image.attrib.get('src')
        idf=int(image.attrib.get('id'))
        print(idf)
        if (idf>=0):
            print('in')
            if os.path.exists(book +"/" +str(pg)+'.png'):
                img = imread(book +"/" +str(pg)+'.png')
                for zone in image.iter('zone'):
                    c=0
                    for point in zone.iter('point'):
                        if c==0:
                            y1= point.attrib.get('y')
                            x1= point.attrib.get('x')
                        if c==2:
                            y2= point.attrib.get('y')
                            x2= point.attrib.get('x')
                        c=c+1
                    id=int(zone.attrib.get('id'))
                    label=None
                    for segment in root.iter('segment'):
                        tid=int(segment.attrib.get('id'))
                        if tid==id:
                            label= segment[1].text
                    if label is None:
                        label='mislabel'
                        print ('there is a mismatch label')

                    label=label.replace('\\', '')
                    label=label.strip()
                    labels.append(label)
                    if not os.path.exists(root_path + label):
                        os.makedirs(root_path + label)

                    roi = img[int(y1)-pad:int(y2)+pad,int(x1)-pad:int(x2)+pad]
                    if roi.any():
                        direc= root_path + label
                        imsave(direc+'/' + str(idf) +'-'+str(idx)+ '.png', roi)
                        idx = idx + 1
        else:
            continue


parsing manuscript  3157556
781
in
there is a mismatch label
782
in
there is a mismatch label
there is a mismatch label
there is a mismatch label
there is a mismatch label
783
in
there is a mismatch label
there is a mismatch label
there is a mismatch label
there is a mismatch label
there is a mismatch label
784
in
there is a mismatch label
there is a mismatch label
785
in
there is a mismatch label
786
in
there is a mismatch label
787
in
788
in
789
in
there is a mismatch label
790
in
there is a mismatch label
791
in
there is a mismatch label
there is a mismatch label
792
in
there is a mismatch label
there is a mismatch label
793
in
there is a mismatch label
there is a mismatch label
794
in
there is a mismatch label
795
in
there is a mismatch label
796
in
797
in
798
in
there is a mismatch label
there is a mismatch label
there is a mismatch label
there is a mismatch label
there is a mismatch label
799
in
there is a mismatch label
800
in
there is a mismatch label
801
in
there is a mismatch

# Cleanup dataset

In [3]:
import os
import shutil

folder = 'segdata'
# Delete these folders
for classfolder in os.listdir(folder):
    length = len(os.listdir(folder + '/' + classfolder))
    deleted_folders = ["k", "mislabel", "ءء", "ءل", "ءه", "ءية"]
    if length <= 0 or (classfolder in deleted_folders):
        print("deleting"+ classfolder +"...")
        shutil.rmtree(folder + '/' + classfolder)


# special case for ا
images = os.listdir(folder + '/' + 'ا')
images = images[:20]
for image in images:
    print("copying " + image)
    shutil.copy(folder + '/' + 'ا' + '/' + image, folder + '/' + "اا")


## After transferred delete previous one and rename new one
shutil.rmtree(folder + '/' + 'ا')
os.rename(folder + '/' +'اا', folder + '/' + 'ا')

copying 1000-112239.png


FileNotFoundError: [Errno 2] No such file or directory: '1000-112239.png'

# Data augmentation

In [1]:
import numpy as np
import cv2
import os
import random


def homographyAugmentation(img, random_limits = (0.9, 1.1)):
        '''
        Creates an augmentation by computing a homography from three
        points in the image to three randomly generated points
        '''
        y, x = img.shape[:2]
        fx = float(x)
        fy = float(y)
        src_point = np.float32([[fx / 2, fy / 3, ],
                                [2 * fx / 3, 2 * fy / 3],
                                [fx / 3, 2 * fy / 3]])
        random_shift = (np.random.rand(3, 2) - 0.5) * 2 * (random_limits[1] - random_limits[0]) / 2 + np.mean(
            random_limits)
        dst_point = src_point * random_shift.astype(np.float32)
        transform = cv2.getAffineTransform(src_point, dst_point)
        # border_value = 0
        if img.ndim == 3:
            border_value = np.median(np.reshape(img, (img.shape[0] * img.shape[1], -1)), axis=0)
        else:
            border_value = np.median(img)
        warped_img = cv2.warpAffine(img, transform, dsize=(x, y), borderValue=border_value.astype(float))
        return warped_img


folder = 'segdata'
os.chdir(folder)
label=0
for classfolder in os.listdir(os.getcwd()):
    # rename to fix arabic path problem
    l = len(str(label))
    z = 5 - l
    a = ''
    for i in range(z):
        a += '0'
    labelValue = a + str(label)
    os.rename(classfolder, labelValue)

    images = os.listdir(os.getcwd() + '/' + labelValue)
    length = len(images)
    if 0 < length <= 20:
        print("augmenting " + classfolder + " ...")
        for i in range(40):
            randIndex = random.randint(1, length)
            randIndex = randIndex - 1
            imgName = str(images[randIndex])
            img = cv2.imread(os.getcwd() + '/' + labelValue + '/' + imgName)
            if img.any():
                img = homographyAugmentation(img=img)
                cv2.imwrite(os.getcwd() + '/' + labelValue + '/' + "augmented_" + str(i) + '.png', img)
    # rename back to its original label
    os.rename(labelValue, classfolder)
    label = label + 1

augmenting 00235 ...


PermissionError: [Errno 13] Permission denied: 'بطة' -> '00235'

# Split dataset

In [4]:
!pip install split-folders

In [6]:
import os
import numpy as np
import shutil
import random
import splitfolders

# creating train / val /test
root_dir = 'segdata/'
new_root = 'dataset/'
# classes = [folderClass for folderClass in os.listdir(root_dir)]
splitfolders.ratio(root_dir, output=new_root, seed=1337, ratio=(.8, 0.1,0.1))
#
# ## creating partition of the data after shuffeling
#
# for cls in classes:
#     src = root_dir + cls  # folder to copy images from
#     print(src)
#
#     allFileNames = os.listdir(src)
#     np.random.shuffle(allFileNames)
#
#     ## here 0.75 = training ratio , (0.95-0.75) = validation ratio , (1-0.95) =
#     ##training ratio
#     train_FileNames, val_FileNames, test_FileNames = np.split(np.array(allFileNames), [int(len(allFileNames) * 0.75),
#                                                                                        int(len(allFileNames) * 0.95)])
#
#     # #Converting file names from array to list
#
#     train_FileNames = [src + '/' + name for name in train_FileNames]
#     val_FileNames = [src + '/' + name for name in val_FileNames]
#     test_FileNames = [src + '/' + name for name in test_FileNames]
#
#     print('Total images  : ' + cls + ' ' + str(len(allFileNames)))
#     print('Training : ' + cls + ' ' + str(len(train_FileNames)))
#     print('Validation : ' + cls + ' ' + str(len(val_FileNames)))
#     print('Testing : ' + cls + ' ' + str(len(test_FileNames)))
#
#     ## Copy pasting images to target directory
#
#     for name in train_FileNames:
#         shutil.copy(name, new_root + 'train/' + cls)
#
#     for name in val_FileNames:
#         shutil.copy(name, new_root + 'val/' + cls)
#
#     for name in test_FileNames:
#         shutil.copy(name, new_root + 'test/' + cls)

Copying files: 166889 files [12:33:32,  3.69 files/s]
